In [1]:
%pwd

'c:\\Users\\nishz\\Downloads\\medic.ai-main\\research'

In [2]:
import os
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

ModuleNotFoundError: No module named 'langchain'

In [ ]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls = PyPDFLoader)
    docs = loader.load()
    return docs

In [ ]:
extracted_data = load_pdf(data='..//Data/')

In [ ]:
# extracted_data

# Chunking : 
def text_split(data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(data)
    return text_chunks

text_chunks = text_split(extracted_data)
print(f"length of chunks : {len(text_chunks)}")

In [ ]:
import torch
print(torch.cuda.get_device_name(0))

In [ ]:
# %pip uninstall bitsandbytes

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings 

def download_hf_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hf_embeddings()
print(embeddings)

In [ ]:
test = embeddings.embed_query("hey there")
print(test)
print(f"Length : {len(test)}") # 384 for 384 dimensions

In [ ]:
!pip show pinecone

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone 
from pinecone import ServerlessSpec
from dotenv import load_dotenv
import os
# Run only once to create index

load_dotenv()

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key = PINECONE_API_KEY)

index_name = "tanishka"

pc.create_index(
    name=index_name,
    dimension= 384, 
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [ ]:
from langchain_pinecone import PineconeVectorStore

index_name = "tanishka"

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name= index_name,
    embedding= embeddings
)


In [ ]:
from dotenv import load_dotenv
load_dotenv()

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key = PINECONE_API_KEY)

index_name = "tanishka"
index = pc.Index(index_name)
# docsearch = index.load(index_name)

In [ ]:
type(index)

In [ ]:
# Above cell will store embeddings of every single chunk of our document.
index.describe_index_stats()

In [ ]:
# index_name = "tanishka"
# pc = Pinecone(api_key = PINECONE_API_KEY)
# docsearch = pc.from_existing_index(
#     name=index_name,
#     dimension= 384, 
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )
# docsearch


In [ ]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [ ]:
docsearch.similarity_search("What is the side effect of paracetamol?")
docsearch.similarity_search("What is Acne, what causes acne ?")

In [ ]:
retriever = docsearch.as_retriever()
# retriever.invoke("What is the side effect of paracetamol?")
retriever.invoke("What is Acne, what causes acne ?")

In [ ]:
docsearch.similarity_search("What is the side effect of paracetamol?")

# Local LLMs For RAG Via LM Studio

In [ ]:
# !pip install openai

In [ ]:
from openai import OpenAI 

client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="gemma-2-2b-instruct",
  messages=[
    {"role": "system", "content": "You are a medical expert named Dr. Medic, You are not a doctor or a medical professor, but you know a lot about medicine, you can assume yourself as a medical wizard/expert,you are given a question and you need to answer it based on the provided context."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

# print(completion.choices[0].message)

In [ ]:
import sys
import time

def animated_print(text, delay=0.05):
    for char in text:
        sys.stdout.write(char)  
        sys.stdout.flush()      
        time.sleep(delay)       

generated_text = completion.choices[0].message.content

# animated_print(generated_text)

In [ ]:
# !pip install langchain-qdrant

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_community.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import pinecone

# Initialize Pinecone
# pinecone.init(
#     api_key="your-pinecone-api-key",
#     environment="your-environment"
# )

# Initialize local embeddings using a sentence transformer model
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",  # This is a lightweight but effective model
    model_kwargs={'device': 'cpu'}
)

# Initialize the vector store with local embeddings
index_name = "tanishka"
vectorstore = Pinecone.from_existing_index(index_name, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Initialize the LangChain wrapper for your local LLM
llm = OpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    model_name="gemma-2-2b-instruct",
    temperature=0.7
)

# Define the prompt template
template = """
You are Dr. Kiri, A medical expert and an assistant for question-answering tasks.
You are given a question and you need to answer it based on the retrieved context.
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
Answer sufficiently in about 100 words and be precise.

Context: {context}
Question: {question}
Answer:
"""

rag_prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the RAG chain
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

def enter_question():
    try:
        print("About to invoke the rag_chain")
        question = input("Enter your prompt: ")
        for chunk in rag_chain.stream(question):
            print(chunk, end="", flush=True)
        print("\nJust finished invoking the rag_chain")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Main loop
if __name__ == "__main__":
    while True:
        enter_question()

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are Dr. Kiri, A medical expert and an assistant for question-answering tasks, "
    "you are given a question and you need to answer it based on the retrieved context to answer."
    "If you don't know the answer, just say that you don't know. Don't try to make up an answer."
    "Answer concisely."
    "\n\n Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{question}")
    ]
)


In [ ]:
type(client)